## Libraries 

In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display



# <font color=blue>*Data Preprocessing*</font>

## Import Data 

In [85]:
#read the .csv file
#change the data-types for columns such as "InvoiceNo", "CustomerID"
df = pd.read_csv("data.csv",encoding='ISO-8859-1',dtype={'InvoiceNo':'str','CustomerID':'str'})
display(df.head())

#convert "InvoiceDate" to datetime object
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print("\nDATA INFORMATION:")
print(df.info())


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom



DATA INFORMATION:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB
None


## <font color=red>**NULL Values**</font>

In [86]:
# find and display null values and % of this in the dataset
df.isnull().sum().values

#dict and dataFrame to diplay null values
nullv_dict = {'Attributes': df.columns,
              'Null Values': df.isnull().sum().values,
              'NaN(%)': (df.isnull().sum().values/df.shape[0])*100}
nullv_df = pd.DataFrame(nullv_dict)

print("\nDATA DIMENSION(row,cols):\n",df.shape)

display(nullv_df)




DATA DIMENSION(row,cols):
 (541909, 8)


,Attributes,Null Values,NaN(%)
0,InvoiceNo,0,0.000000
1,StockCode,0,0.000000
2,Description,1454,0.268311
3,Quantity,0,0.000000
4,InvoiceDate,0,0.000000
5,UnitPrice,0,0.000000
6,CustomerID,135080,24.926694
7,Country,0,0.000000


### <font color="green">Insight-1</font>
- Approximately **25%** entries are not associated with **CustomerID**
- With the data available, it is **not possible** to impute these values 
- So, we will remove them from the dataset

## <font color=red>**Removing NULL Values**</font>

In [87]:
# drop rows with "CustomerID" null 
df.dropna(subset=['CustomerID'],inplace=True)

#dict and dataFrame to diplay null values
nullv_dict = {'Attributes': df.columns,
              'Null Values': df.isnull().sum().values,
              'NaN(%)': (df.isnull().sum().values/df.shape[0])*100}
nullv_df = pd.DataFrame(nullv_dict)

print("\nDATA DIMENSION(row,cols):\n",df.shape)

display(nullv_df)


DATA DIMENSION(row,cols):
 (406829, 8)


,Attributes,Null Values,NaN(%)
0,InvoiceNo,0,0.0
1,StockCode,0,0.0
2,Description,0,0.0
3,Quantity,0,0.0
4,InvoiceDate,0,0.0
5,UnitPrice,0,0.0
6,CustomerID,0,0.0
7,Country,0,0.0


## <font color=red>**Duplicate Entries**</font>

In [88]:
# find duplicate entries if any
print("Total Duplicated Entries(before deletion):",df.duplicated().sum(),"\n")

#remove these 5225 entries from the dataset
df.drop_duplicates(inplace=True)

#check for duplicates
print("Total Duplicated Entries(before deletion):",df.duplicated().sum(),"\n")

print("DATA DIMENSION(row,cols):\n",df.shape)

Total Duplicated Entries(before deletion): 5225 

Total Duplicated Entries(before deletion): 0 

DATA DIMENSION(row,cols):
 (401604, 8)


# <font color=blue>*Exploratory Data Analysis*</font>

### <font color="green">Attributes Classification:</font>
- **InvoiceNo:** **_Nominal_** Six-digit unique number for each Transcation. 'c' stands for cancellation. 
- **StockCode:** **_Nominal_** Five-digit unique number for each distinct product. 
- **Description:** **_Nominal_** Product Name and short Description. 
- **Quantity:** **_Numeric_** Amount of each product per transcation. 
- **InvoiceDate:** **_Numeric_** Date and Time when each transcation occured. 
- **UnitPrice:** **_Numeric_** Product Price in Sterling Pounds per unit. 
- **CustomerID:** **_Nominal_** Five-digit integral number assigned to each customer. 
- **Country:** **_Nominal_** Country Name of Customer Residence. 

## Customer, Product and Invoice Details

In [129]:
print('Total number of Entries:',df.shape[0])

#find number of unique customers, product and invoices
print('\nTotal Unique Customers:',len(df['CustomerID'].value_counts()))
print('Total Unique Products:',len(df['StockCode'].value_counts()))
print('Total Unique Invoices:',len(df['InvoiceNo'].value_counts()))

Total number of Entries: 401604

Total Unique Customers: 4372
Total Unique Products: 3684
Total Unique Invoices: 22190


### <font color="green">Insights-2</font>
- Data consists of **4372** customers, and these users bought **3684** different products.
- Total number of transcations are **22190**.

## Country 

In [126]:
#information countries from where orders took place:
cntry_df = df[['CustomerID','InvoiceNo','Country']]
cntry_df = cntry_df.groupby(['CustomerID','InvoiceNo','Country']).count()
cntry_df = cntry_df.reset_index()

print("Total number of countries:",len(cntry_df['Country'].value_counts().index))

disc_df = pd.DataFrame({'Country':cntry_df['Country'].value_counts().index, 
                        'Number of orders':cntry_df['Country'].value_counts().values})
display(disc_df)

print("Inference: Most of the orders were initiated from United Kingdom.")

Total number of countries: 37


,Country,Number of orders
0,United Kingdom,19857
1,Germany,603
2,France,458
3,EIRE,319
4,Belgium,119
5,Spain,105
6,Netherlands,101
7,Switzerland,71
8,Portugal,70
9,Australia,69


Inference: Most of the orders were initiated from United Kingdom.
